<a href="https://colab.research.google.com/github/sutharimanikanta/Golab/blob/main/Brain_Tumor(multi_class_classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from keras.models import Sequential,Model
from keras.layers import Input,Dense,GlobalAveragePooling2D, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,TensorBoard,LambdaCallback
from keras import models
from keras import layers
import tensorflow as tf
import os
import os.path
from pathlib import Path
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical#matrix where
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.utils import shuffle
from keras import regularizers
from tensorflow.keras.optimizers import RMSprop,Adam

from tensorflow.keras.applications.resnet50 import ResNet50


In [ ]:
filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning)
filterwarnings("ignore", category=UserWarning)

In [ ]:
!git clone https://github.com/SartajBhuvaji/Brain-Tumor-Classification-DataSet

Cloning into 'Brain-Tumor-Classification-DataSet'...
remote: Enumerating objects: 3039, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 3039 (delta 0), reused 0 (delta 0), pack-reused 3035
Receiving objects: 100% (3039/3039), 79.25 MiB | 29.98 MiB/s, done.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Set the seed
tf.random.set_seed(42)

# Create ImageDataGenerator instances for train and validation data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    shear_range=0.1,  # set range for random shear
    zoom_range=0.1,  # set range for random zoom
    horizontal_flip=True,  # randomly flip images
    fill_mode='nearest'
                                  )
valid_datagen = ImageDataGenerator(rescale=1./255)

# Define the directories for train and validation data
train_dir = "/content/Brain-Tumor-Classification-DataSet/Training"
valid_dir = "/content/Brain-Tumor-Classification-DataSet/Testing"

# Flow from directory for train and validation data
train_data = train_datagen.flow_from_directory(train_dir,
                                                batch_size=32, # number of images to process at a time
                                               target_size=(224, 224), # convert all images to be 224 x 224
                                               class_mode="categorical", # type of problem we're working on

                                               )
valid_data = valid_datagen.flow_from_directory(valid_dir,
                                                batch_size=32, # number of images to process at a time
                                               target_size=(224, 224), # convert all images to be 224 x 224
                                               class_mode="categorical",
                                               )


Found 2870 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dropout  # Import the Dropout layer

# Assuming you have 4 classes
num_classes = 4

# Define the model
model_multi_class = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (2, 2), activation="relu"),
    tf.keras.layers.Conv2D(64, (2, 2), activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding="valid"),
    Dropout(0.1),  # Corrected spelling and imported Dropout layer
    tf.keras.layers.Conv2D(32, (2, 2), activation="relu"),
    tf.keras.layers.Conv2D(32, (2, 2), activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding="valid"),
    Dropout(0.1),  # Corrected spelling
    tf.keras.layers.Conv2D(16, (2, 2), activation="relu"),
    tf.keras.layers.Conv2D(16, (2, 2), activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding="valid"),
    Dropout(0.1),  # Corrected spelling
    tf.keras.layers.Conv2D(8, (2, 2), activation="relu"),
    tf.keras.layers.Conv2D(8, (2, 2), activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding="valid"),
    Dropout(0.1),  # Corrected spelling
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation="relu"),
    Dropout(0.2),  # Added Dropout layer after the Dense layer
    tf.keras.layers.Dense(num_classes, activation="softmax")  # Output layer for multi-class classification
])

# Compile the model
model_multi_class.compile(loss="categorical_crossentropy",  # Categorical crossentropy loss for multi-class
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=["accuracy"])

# Fit the model
history_multi_class = model_multi_class.fit(train_data,
                                            epochs=5,
                                            steps_per_epoch=len(train_data),
                                            validation_data=valid_data,
                                            validation_steps=len(valid_data))


Epoch 1/5
90/90 [==============================] - 58s 532ms/step - loss: 1.2149 - accuracy: 0.4425 - val_loss: 1.6668 - val_accuracy: 0.2386
Epoch 2/5
90/90 [==============================] - 49s 544ms/step - loss: 1.0097 - accuracy: 0.5690 - val_loss: 1.9189 - val_accuracy: 0.3452
Epoch 3/5
90/90 [==============================] - 46s 506ms/step - loss: 0.8919 - accuracy: 0.6223 - val_loss: 1.8870 - val_accuracy: 0.3249
Epoch 4/5
90/90 [==============================] - 44s 492ms/step - loss: 0.8834 - accuracy: 0.6240 - val_loss: 2.0975 - val_accuracy: 0.2614
Epoch 5/5
90/90 [==============================] - 45s 498ms/step - loss: 0.8178 - accuracy: 0.6526 - val_loss: 1.8846 - val_accuracy: 0.3604


In [ ]:
loss, accuracy= model_multi_class.evaluate(valid_data)
print(f"loss is:{loss}")
print(f"accuacy is:{accuracy}")

13/13 [==============================] - 1s 85ms/step - loss: 1.8846 - accuracy: 0.3604
loss is:1.8845906257629395
accuacy is:0.3604061007499695


(None, None, None, 64) ->4 dim
* 1none->batch size,
* 1none->feature map,
* 1none->feature map,


In [ ]:
model_multi_class.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, None, None, 64)    832       
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 64)    16448     
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 64)    0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, None, None, 64)    0         
                                                                 
 conv2d_2 (Conv2D)           (None, None, None, 32)    8224      
                                                                 
 conv2d_3 (Conv2D)           (None, None, None, 32)    4128      
                                                        

In [ ]:
# Create checkpoint callback to save model for later use
checkpoint_path = "101_classes_10_percent_data_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True, # save only the model weights
                                                         monitor="val_accuracy", # save the model weights which score the best validation accuracy
                                                         save_best_only=True) # only keep the best model weights on file (delete the rest)


In [ ]:
Modifed = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (2, 2), activation="relu"),
    tf.keras.layers.Conv2D(64, (2, 2), activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding="valid"),
    Dropout(0.1),  # Corrected spelling and imported Dropout layer
    tf.keras.layers.Conv2D(32, (2, 2), activation="relu"),
    tf.keras.layers.Conv2D(32, (2, 2), activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding="valid"),
    Dropout(0.1),  # Corrected spelling
    tf.keras.layers.Conv2D(16, (2, 2), activation="relu"),
    tf.keras.layers.Conv2D(16, (2, 2), activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding="valid"),
    Dropout(0.1),  # Corrected spelling
    tf.keras.layers.Conv2D(8, (2, 2), activation="relu"),
    tf.keras.layers.Conv2D(8, (2, 2), activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding="valid"),
    Dropout(0.1),  # Corrected spelling
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation="relu"),
    Dropout(0.2),  # Added Dropout layer after the Dense layer
    tf.keras.layers.Dense(num_classes, activation="softmax")  # Output layer for multi-class classification
])

# Compile the model
model_multi_class.compile(loss="categorical_crossentropy",  # Categorical crossentropy loss for multi-class
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=["accuracy"])

# Fit the model
history_multi_class = model_multi_class.fit(train_data,
                                            epochs=5,
                                            steps_per_epoch=len(train_data),
                                            validation_data=valid_data,
                                            validation_steps=len(valid_data))
